# Apache Pulsar - Architecture Overview
[Apache Pulsar 2.7.1 - Architecture Overview](https://pulsar.apache.org/docs/en/concepts-architecture-overview/)

At the highest level, a Pulsar instance is composed of one or more Pulsar clusters. Clusters within an instance can replicate data amongst themselves.\
在最高水平上，一个Pulsar实例是由一个或多个Pulsar集群组成的。一个实例中的集群可以在它们自己之间复制数据。

In a Pulsar cluster:
* One or more brokers handles and load balances incoming messages from producers, dispatches messages to consumers, communicates with the Pulsar configuration store to handle various coordination tasks, stores messages in BookKeeper instances (aka bookies), relies on a cluster-specific ZooKeeper cluster for certain tasks, and more.\
一个或多个broker处理和负载平衡来自生产者的传入消息，将消息分派给消费者，与Pulsar configuration store通信以处理各种协调任务，将消息存储在BookKeeper实例（即bookies）中，某些任务依赖于特定集群的ZooKeeper集群，等等。
* A BookKeeper cluster consisting of one or more bookies handles persistent storage of messages.\
由一个或多个bookie组成的BookKeeper集群处理消息的持久存储。
* A ZooKeeper cluster specific to that cluster handles coordination tasks between Pulsar clusters.\
特定于该集群的ZooKeeper集群处理Pulsar集群之间的协调任务。

The diagram below provides an illustration of a Pulsar cluster:\
下图展示了一个Pulsar集群：
<img src = "https://pulsar.apache.org/docs/assets/pulsar-system-architecture.png">

At the broader instance level, an instance-wide ZooKeeper cluster called the configuration store handles coordination tasks involving multiple clusters, for example [geo-replication](https://pulsar.apache.org/docs/en/concepts-replication).\
在更广泛的实例级别，称为配置存储的实例范围的ZooKeeper集群处理涉及多个集群的协调任务，例如[geo-replication](https://pulsar.apache.org/docs/en/concepts-replication)。

## Brokers

The Pulsar message broker is a stateless component that's primarily responsible for running two other components:\
Pulsar message broker是一个无状态组件，主要负责运行其他两个组件:
* An HTTP server that exposes a REST API for both administrative tasks and topic lookup for producers and consumers\
一个HTTP服务器，它为管理任务和为生产者和消费者公开一个主题查找的REST API
* A dispatcher, which is an asynchronous TCP server over a custom binary protocol used for all data transfers\
调度器，它是一个异步TCP服务器，使用用于所有数据传输的自定义二进制协议

Messages are typically dispatched out of a managed ledger cache for the sake of performance, unless the backlog exceeds the cache size. If the backlog grows too large for the cache, the broker will start reading entries from BookKeeper.\
出于性能考虑，消息通常从托管分类账缓存中分派，除非积压超过缓存大小。如果积压增长到超出缓存的范围，代理将开始从簿记员读取条目。

Finally, to support geo-replication on global topics, the broker manages replicators that tail the entries published in the local region and republish them to the remote region using the Pulsar Java client library.\
最后，为了支持全局主题上的geo-replication，broker管理跟踪在本地区域发布的条目的复制器，并使用Pulsar Java客户端库将它们重新发布到远程区域。

For a guide to managing Pulsar brokers, see the [brokers](https://pulsar.apache.org/docs/en/admin-api-brokers) guide.\
有关Pulsar broker管理的指南，请参见[brokers](https://pulsar.apache.org/docs/en/admin-api-brokers)指南。

## Clusters

A Pulsar instance consists of one or more Pulsar clusters. Clusters, in turn, consist of:\
一个Pulsar实例由一个或多个Pulsar集群组成。集群依次由:
* One or more Pulsar brokers\
一个或多个Pulsar broker
* A ZooKeeper quorum used for cluster-level configuration and coordination\
用于集群级别的配置和协调的ZooKeeper quorum
* An ensemble of bookies used for persistent storage of messages\
用于持久存储信息的一组bookie

Clusters can replicate amongst themselves using [geo-replication](https://pulsar.apache.org/docs/en/concepts-replication).\
集群可以使用[geo-replication](https://pulsar.apache.org/docs/en/concepts-replication)在自身之间进行复制。

For a guide to managing Pulsar clusters, see the [clusters](https://pulsar.apache.org/docs/en/admin-api-clusters) guide.\
有关管理Pulsar集群的指南，请参见Pulsar[集群](https://pulsar.apache.org/docs/en/admin-api-clusters)指南。

## Metadata store

Pulsar uses [Apache Zookeeper](https://zookeeper.apache.org/) for metadata storage, cluster configuration, and coordination. In a Pulsar instance:\
Pulsar使用[Apache Zookeeper](https://zookeeper.apache.org/)进行元数据存储、集群配置和协调。在一个Pulsar实例中:
* A configuration store quorum stores configuration for tenants, namespaces, and other entities that need to be globally consistent.\
配置存储仲裁为租户、命名空间和其他需要全局一致的实体存储配置。
* Each cluster has its own local ZooKeeper ensemble that stores cluster-specific configuration and coordination such as which brokers are responsible for which topics as well as ownership metadata, broker load reports, BookKeeper ledger metadata, and more.\
每个集群都有自己的local ZooKeeper集成，存储集群特定的配置和协调，比如哪个代理负责哪个主题，所有权元数据，代理负载报告，簿记记账元数据等等。

## Service discovery

[Clients](https://pulsar.apache.org/docs/en/client-libraries) connecting to Pulsar brokers need to be able to communicate with an entire Pulsar instance using a single URL. Pulsar provides a built-in service discovery mechanism that you can set up using the instructions in the [Deploying a Pulsar instance](https://pulsar.apache.org/docs/en/deploy-bare-metal#service-discovery-setup) guide.\
[Clients](https://pulsar.apache.org/docs/en/client-libraries)连接到Pulsar broker需要能够通过一个URL与整个Pulsar实例通信。Pulsar提供了一种内置的服务发现机制，您可以使用[部署一个Pulsar实例](https://pulsar.apache.org/docs/en/deploy-bare-metal#service-discovery-setup)指南中的说明进行设置。

You can use your own service discovery system if you'd like. If you use your own system, there is just one requirement: when a client performs an HTTP request to an endpoint, such as `http://pulsar.us-west.example.com:8080`, the client needs to be redirected to some active broker in the desired cluster, whether via DNS, an HTTP or IP redirect, or some other means.\
如果愿意，您可以使用自己的服务发现系统。如果你使用自己的系统，只有一个要求：当client执行HTTP请求，比如`http://pulsar.us-west.example.com:8080`，client需要被重定向到一些在所需集群中活跃的broker，无论是通过DNS、HTTP或IP重定向或其他手段。

The diagram below illustrates Pulsar service discovery:
<img src = "https://pulsar.apache.org/docs/assets/pulsar-service-discovery.png">

In this diagram, the Pulsar cluster is addressable via a single DNS name: `pulsar-cluster.acme.com`. A [Python client](https://pulsar.apache.org/docs/en/client-libraries-python), for example, could access this Pulsar cluster like this:\
在这张图中，这个Pulsar集群可以通过一个单独的DNS名称来寻址：`pulsar-cluster.acme.com`。例如，一个[Python client](https://pulsar.apache.org/docs/en/client-libraries-python)可以这样访问这个Pulsar集群：
```python
from pulsar import Client
client = Client('pulsar://pulsar-cluster.acme.com:6650')
```

__Note__\
In Pulsar, each topic is handled by only one broker. Initial requests from a client to read, update or delete a topic are sent to a broker that may not be the topic owner. If the broker cannot handle the request for this topic, it redirects the request to the appropriate broker.\
在Pulsar中，每个主题只由一个broker处理。client读取、更新或删除主题的初始请求被发送给可能不是主题所有者的broker。如果broker不能处理此主题的请求，则将请求重定向到适当的broker。